In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
necoc = 10
generations = int(3000/necoc)
generations

300

In [3]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [4]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

In [5]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [6]:
import numpy as np
import random

number_codes = []
for i in range(2**(26-1), 2**26-1):
    if (i % 1000000) == 0:
            print(i)
    for j in bin(i).split('b')[1]:
        number_codes.append(int(j))
        


number_codes = np.array(number_codes).reshape(2**(26 - 1) - 1,26).T     

slice = random.sample(range(2**(26 - 1) - 1), necoc)

number_codes = number_codes[:,slice].tolist()

classes = []
for i in range(necoc):
    code_list = np.array(number_codes).T[i]
    classes.append(np.where(code_list==1)[0].tolist())

matrix_errors = []

for i in range(25):
    for j in range(i+1,26):
        matrix_errors.append(necoc - np.sum(np.array(number_codes[i]) == np.array(number_codes[j])))
min_error = np.min(matrix_errors)

print(min_error)

34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
1


In [7]:
winner_list = []

for i,class1 in enumerate(classes):
    print("running:{}/30".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/30
number of class1:864
number of class2:736

 ****** Running generation 0 ****** 

Population's average fitness: 0.49123 stdev: 0.04436
Best fitness: 0.58313 - size: (2, 3) - species 3 - id 31
Average adjusted fitness: 0.117
Mean genetic distance 2.734, standard deviation 0.516
Population of 199 members in 13 species
Total extinctions: 0
Generation time: 4.140 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.51584 stdev: 0.04383
Best fitness: 0.61062 - size: (3, 4) - species 6 - id 267
Average adjusted fitness: 0.124
Mean genetic distance 2.743, standard deviation 0.549
Population of 200 members in 14 species
Total extinctions: 0
Generation time: 3.381 sec (3.760 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.51945 stdev: 0.04951
Best fitness: 0.63625 - size: (3, 6) - species 1 - id 500
Average adjusted fitness: 0.141
Mean genetic distance 2.792, standard deviation 0.563
Population of 201 members in 16 species
Total 

Mean genetic distance 3.240, standard deviation 0.512
Population of 199 members in 31 species
Total extinctions: 0
Generation time: 5.500 sec (4.955 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.55132 stdev: 0.06282
Best fitness: 0.65563 - size: (6, 10) - species 27 - id 3332
Average adjusted fitness: 0.119
Mean genetic distance 3.252, standard deviation 0.497
Population of 196 members in 31 species
Total extinctions: 0
Generation time: 5.202 sec (4.951 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.55629 stdev: 0.06051
Best fitness: 0.65563 - size: (6, 10) - species 27 - id 3332
Average adjusted fitness: 0.135
Mean genetic distance 3.241, standard deviation 0.509
Population of 196 members in 30 species
Total extinctions: 0
Generation time: 5.246 sec (5.009 average)

 ****** Running generation 27 ****** 

Population's average fitness: 0.54771 stdev: 0.06612
Best fitness: 0.65625 - size: (5, 9) - species 27 - id 3881
A

Mean genetic distance 3.278, standard deviation 0.541
Population of 203 members in 20 species
Total extinctions: 0
Generation time: 5.232 sec (5.735 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.54699 stdev: 0.06287
Best fitness: 0.65938 - size: (7, 12) - species 27 - id 5194
Average adjusted fitness: 0.127
Mean genetic distance 3.267, standard deviation 0.527
Population of 203 members in 20 species
Total extinctions: 0
Generation time: 5.260 sec (5.608 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.54131 stdev: 0.06385
Best fitness: 0.65938 - size: (7, 12) - species 27 - id 5194
Average adjusted fitness: 0.135
Mean genetic distance 3.284, standard deviation 0.522
Population of 201 members in 20 species
Total extinctions: 0
Generation time: 5.113 sec (5.517 average)

 ****** Running generation 51 ****** 

Population's average fitness: 0.54920 stdev: 0.06119
Best fitness: 0.65938 - size: (7, 12) - species 27 - id 5194


Population's average fitness: 0.55629 stdev: 0.06690
Best fitness: 0.66375 - size: (8, 14) - species 27 - id 8614
Average adjusted fitness: 0.149
Mean genetic distance 3.096, standard deviation 0.589
Population of 197 members in 15 species
Total extinctions: 0
Generation time: 5.562 sec (5.625 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.55045 stdev: 0.07385
Best fitness: 0.66375 - size: (8, 14) - species 27 - id 8614
Average adjusted fitness: 0.145
Mean genetic distance 3.102, standard deviation 0.586
Population of 198 members in 15 species
Total extinctions: 0
Generation time: 5.669 sec (5.628 average)

 ****** Running generation 75 ****** 

Population's average fitness: 0.55531 stdev: 0.07049
Best fitness: 0.66375 - size: (8, 14) - species 27 - id 8614
Average adjusted fitness: 0.132
Mean genetic distance 3.113, standard deviation 0.575
Population of 199 members in 17 species
Total extinctions: 0
Generation time: 5.729 sec (5.607 average)

 ******

Population's average fitness: 0.56677 stdev: 0.06683
Best fitness: 0.66375 - size: (8, 14) - species 27 - id 8614
Average adjusted fitness: 0.191
Mean genetic distance 3.007, standard deviation 0.530
Population of 200 members in 21 species
Total extinctions: 0
Generation time: 5.529 sec (5.257 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.56100 stdev: 0.07495
Best fitness: 0.66375 - size: (8, 14) - species 27 - id 8614
Average adjusted fitness: 0.188
Mean genetic distance 3.000, standard deviation 0.531
Population of 200 members in 21 species
Total extinctions: 0
Generation time: 5.367 sec (5.304 average)

 ****** Running generation 99 ****** 

Population's average fitness: 0.55986 stdev: 0.07279
Best fitness: 0.66375 - size: (8, 14) - species 27 - id 8614
Average adjusted fitness: 0.180
Mean genetic distance 3.009, standard deviation 0.554
Population of 193 members in 19 species
Total extinctions: 0
Generation time: 5.400 sec (5.333 average)

 ******

...

Population's average fitness: 0.54431 stdev: 0.06604
Best fitness: 0.65875 - size: (7, 16) - species 18 - id 22245
Average adjusted fitness: 0.120
Mean genetic distance 2.654, standard deviation 0.559
Population of 200 members in 7 species
Total extinctions: 0
Generation time: 15.345 sec (16.187 average)

 ****** Running generation 204 ****** 

Population's average fitness: 0.55088 stdev: 0.06219
Best fitness: 0.65875 - size: (7, 16) - species 18 - id 22245
Average adjusted fitness: 0.134
Mean genetic distance 2.662, standard deviation 0.552
Population of 198 members in 7 species
Total extinctions: 0
Generation time: 14.886 sec (15.551 average)

 ****** Running generation 205 ****** 

Population's average fitness: 0.53784 stdev: 0.06974
Best fitness: 0.65875 - size: (7, 16) - species 18 - id 22245
Average adjusted fitness: 0.113
Mean genetic distance 2.696, standard deviation 0.559
Population of 200 members in 7 species
Total extinctions: 0
Generation time: 17.061 sec (15.624 average)


Population's average fitness: 0.55713 stdev: 0.05943
Best fitness: 0.65938 - size: (22, 46) - species 103 - id 37676
Average adjusted fitness: 0.123
Mean genetic distance 2.966, standard deviation 0.509
Population of 200 members in 21 species
Total extinctions: 0
Generation time: 14.029 sec (12.829 average)

 ****** Running generation 228 ****** 

Population's average fitness: 0.55582 stdev: 0.06227
Best fitness: 0.65938 - size: (22, 46) - species 103 - id 37676
Average adjusted fitness: 0.141
Mean genetic distance 3.001, standard deviation 0.534
Population of 200 members in 21 species
Total extinctions: 0
Generation time: 12.706 sec (12.584 average)

 ****** Running generation 229 ****** 

Population's average fitness: 0.55706 stdev: 0.06347
Best fitness: 0.65938 - size: (22, 46) - species 103 - id 37676
Average adjusted fitness: 0.133
Mean genetic distance 2.993, standard deviation 0.522
Population of 201 members in 21 species
Total extinctions: 0
Generation time: 12.052 sec (12.501 

Mean genetic distance 2.886, standard deviation 0.538
Population of 202 members in 13 species
Total extinctions: 0
Generation time: 14.065 sec (13.819 average)

 ****** Running generation 251 ****** 

Population's average fitness: 0.55970 stdev: 0.06338
Best fitness: 0.65938 - size: (22, 46) - species 103 - id 37676
Average adjusted fitness: 0.144
Mean genetic distance 2.909, standard deviation 0.537
Population of 202 members in 14 species
Total extinctions: 0
Generation time: 14.671 sec (13.977 average)

 ****** Running generation 252 ****** 

Population's average fitness: 0.54974 stdev: 0.06545
Best fitness: 0.65938 - size: (22, 46) - species 103 - id 37676
Average adjusted fitness: 0.143
Mean genetic distance 2.764, standard deviation 0.514
Population of 199 members in 12 species
Total extinctions: 0
Generation time: 15.914 sec (14.172 average)

 ****** Running generation 253 ****** 

Population's average fitness: 0.55448 stdev: 0.06327
Best fitness: 0.65938 - size: (22, 46) - speci

Population's average fitness: 0.55160 stdev: 0.06755
Best fitness: 0.66000 - size: (22, 45) - species 136 - id 44486
Average adjusted fitness: 0.125
Mean genetic distance 2.843, standard deviation 0.545
Population of 201 members in 13 species
Total extinctions: 0
Generation time: 14.730 sec (14.002 average)

 ****** Running generation 275 ****** 

Population's average fitness: 0.55485 stdev: 0.06621
Best fitness: 0.66250 - size: (23, 44) - species 137 - id 46012
Average adjusted fitness: 0.134
Mean genetic distance 2.937, standard deviation 0.537
Population of 199 members in 12 species
Total extinctions: 0
Generation time: 14.286 sec (14.036 average)

 ****** Running generation 276 ****** 

Population's average fitness: 0.54538 stdev: 0.06914
Best fitness: 0.66250 - size: (23, 44) - species 137 - id 46012
Average adjusted fitness: 0.152
Mean genetic distance 2.922, standard deviation 0.539
Population of 200 members in 12 species
Total extinctions: 0
Generation time: 15.540 sec (14.127 

Mean genetic distance 3.020, standard deviation 0.567
Population of 194 members in 16 species
Total extinctions: 0
Generation time: 16.451 sec (16.146 average)

 ****** Running generation 298 ****** 

Population's average fitness: 0.54789 stdev: 0.06844
Best fitness: 0.66250 - size: (23, 44) - species 140 - id 46012
Average adjusted fitness: 0.145
Mean genetic distance 3.032, standard deviation 0.595
Population of 205 members in 15 species
Total extinctions: 0
Generation time: 18.376 sec (16.322 average)

 ****** Running generation 299 ****** 

Population's average fitness: 0.55361 stdev: 0.06692
Best fitness: 0.66625 - size: (23, 47) - species 136 - id 50235
Average adjusted fitness: 0.154
Mean genetic distance 3.041, standard deviation 0.586
Population of 199 members in 15 species
Total extinctions: 0
Generation time: 22.684 sec (17.005 average)


In [8]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(necoc - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.2275

In [9]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(26 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.275
Avg Base Learner:0.7118749999999999


In [10]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:239 Total connections::2453
